# Optimización de hiperparámetros con KerasTuner

Vamos a utilizar el Dataset MNIST

In [3]:
import keras
from keras import layers
import keras_tuner
from keras_tuner.tuners import RandomSearch

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the images
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# Convert labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

Definimos el modelo. Vamos a definir los siguientes hiperparametros tuneables:

Número de capas de la red: Numero entero entre 1 y 3
Neuronas de cada capa: Número entero entre 32 y 512, en incrementos de 32
Tasa de entrenamiento: 0.001,0.0001,0,00001

In [ ]:

# Define the model
def build_model(hp):
    model = keras.Sequential()
    
    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
            activation='relu'
        ))
    
    # Output layer
    model.add(layers.Dense(10, activation='softmax'))
    
    # Tune the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Set up the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='mnist_ann_tuning'
)

# Perform the search
tuner.search(x_train, y_train, epochs=5, validation_split=0.2)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
loss, accuracy = best_model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Summary of the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)

Trial 5 Complete [00h 03m 06s]
val_accuracy: 0.9524722099304199

Best val_accuracy So Far: 0.9769999980926514
Total elapsed time: 00h 13m 23s
 49/313 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9823 - loss: 0.0598

c:\Users\igome\anaconda3\envs\deepl\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9767 - loss: 0.0831
Test Loss: 0.07373062521219254
Test Accuracy: 0.9790999889373779
{'num_layers': 1, 'units_0': 288, 'learning_rate': 0.001}
